In [2]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_openscale import APIClient as wosclient
from ibm_watson_studio_pipelines import WSPipelines
from botocore.client import Config
import ibm_boto3
import pandas as pd
import json
import os
import requests

In [3]:
CLOUD_API_KEY = os.getenv("cloud_api_key")
space_id = os.getenv("space_id")
pre_prod_model_id = os.getenv('pre_prod_model_id')
data_mart_id = os.getenv("data_mart_id")

## If the model deploys to prod env, check if the model is approved for production

In [4]:
authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
wos_client = wosclient(authenticator=authenticator,service_instance_id=data_mart_id)
wos_client.version

ValueError: The apikey shouldn't be None.

#### Check if the deployment env is prod env

In [8]:
service_providers = wos_client.service_providers.list().result.service_providers

Check the tag for the service provider

In [9]:
PROD_DEPLOYMENT = False
for service_provider in service_providers:
    deployment_space_id = service_provider.entity.deployment_space_id
    if deployment_space_id == space_id and service_provider.entity.operational_space_id == "production":
        PROD_DEPLOYMENT = True
print('The deployed model is in production environment:', PROD_DEPLOYMENT)

The deployed model is in production environment: True


#### Check in Openscale if the model is approved for production 

In [10]:
def check_prod_approval(data_mart_id, wos_client):
    subscriptions = wos_client.subscriptions.list().result.subscriptions
    for subscription in subscriptions:
        sub_model_id = subscription.entity.asset.asset_id
        if sub_model_id == pre_prod_model_id:
            if subscription.entity.risk_evaluation_status.state == "approved":
                return True
            else:
                return False
    return False

In [11]:
if PROD_DEPLOYMENT:
    PREPROD_MODEL_APPROVED = check_prod_approval(data_mart_id, wos_client)
    print('The pre-prod model was approved for production: ', PREPROD_MODEL_APPROVED)

The pre-prod model was approved for production:  True


### Output to pipeline

In [12]:
deployment_prod_checks = {}
deployment_prod_checks['prod_deployment'] = PROD_DEPLOYMENT
deployment_prod_checks['preprod_model_approval'] = PREPROD_MODEL_APPROVED

In [14]:
pipelines_client = WSPipelines.from_apikey(apikey=CLOUD_API_KEY)
pipelines_client.store_results(deployment_prod_checks)

Running outside of Watson Studio Pipeline - storing results in the local filesystem for testing purposes...

  output paths:
    - "prod_deployment": .ibm_watson_studio_pipelines/results/prod_deployment
    - "preprod_model_approval": .ibm_watson_studio_pipelines/results/preprod_model_approval
